<a href="https://colab.research.google.com/github/gaganmathur99/agentic-system/blob/main/multiagent_notebook_smol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# This Notebook creates a Article writer Agent



In [4]:
!pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q gradio_client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 8.6 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()

For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [19]:
os.environ["SERPAPI_API_KEY"] = userdata.get('SERPAPI_API_KEY')

In [38]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, HfApiModel, VisitWebpageTool, DuckDuckGoSearchTool
from google.colab import userdata

model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096)

In [22]:
# Google Research agent to find relevant content
research_agent = CodeAgent(
    model=model,
    tools=[
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
    ],
    name="research_agent",
    description="Browses the web to find information about a particular topic.",
    verbosity_level=0,
    max_steps=10,
)

In [41]:
from smolagents import CodeAgent, HfApiModel, Tool
image_generation_tool = Tool.from_space(
    "black-forest-labs/FLUX.1-schnell",
    name="image_generator",
    description="Generate an image from a prompt"
)


Loaded as API: https://black-forest-labs-flux-1-schnell.hf.space ✔


In [43]:
manager_agent = CodeAgent(
    model=model,
    tools=[image_generation_tool],
    managed_agents=[research_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
        "IPython"
    ],
    planning_interval=5,
    verbosity_level=2,
    max_steps=10,
)

In [44]:
manager_agent.run("""
Find the most trending research about the new matter discovered by Microsoft, combine the information to write an amazing article on it that can be published on the website.
Structure the article professionally, proofread the article and iterate to improve the structure. The article should be interesting and detailed.
Create a cover image for the article. And then display the article and image together.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find the most trending research about the new matter discovered by Microsoft, combine the information to write  │
│ an amazing article on it that can be published on the website.                                                  │
│ Structure the article professionally, proofread the article and iterate to improve the structure. The article   │
│ should be interesting and detailed.                                                                             │
│ Create a cover image for the article. And then display the article and image together.                          │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

[Step 0: Duration 0.09 seconds]

HfHubHTTPError: 402 Client Error: Payment Required for url: https://api-inference.huggingface.co/models/Qwen/Qwen2.5-Coder-32B-Instruct/v1/chat/completions (Request ID: Root=1-67c0f6cc-4045e12c0b9bd6c9633334ea;a06b5b0d-93a6-49b2-9ff0-e18938819088)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly allowance.

In [32]:
from IPython.display import Image as IPImage, display
from IPython.display import FileLink

# Assuming the image is saved as /tmp/gradio/7f13c86defd743fcf5174e1716392062b36d7c8a4ad5e18ad4be2738b394bf02/image.webp
image_path = '/tmp/gradio/aac9da55f6a6f2e0c2496cd0c157f211f3c11f94da0517ea9475216e4bc5ce9d/image.webp'
try:
    # Check if the file exists
    with open(image_path, 'r'):
      # If the file exists, create a displayable link
      display(FileLink(image_path))
except FileNotFoundError:
    print("Error: saved_cover.png not found. Please make sure the image generation was successful and the file exists.")

/tmp/gradio/aac9da55f6a6f2e0c2496cd0c157f211f3c11f94da0517ea9475216e4bc5ce9d/image.webp